# Particle Swarm

## Description
Particle swarm is a stochastic optimization method based on particles at different positions that simultaneously explore the optimization function and influence each other's search.  

Each particle in the swarm is characterized by the following properties:
* current position
* current velocity
* position of the minimum found by this particle

The rule that each particle uses to update its' next position is based on the following:
* current velocity of the particle
* velocity in the direction of the minimum found by this particle so far
* velocity in the direction of the global minimum found by all particles so far

### Algorithm
1. Initialize a list of $n \times p$-dimensional particles with a random initial position $(x_{1,0}, \cdots, x_{n,0})$ and random velocity $(v_{1,0}, \cdots, v_{n,0})$.
2. For each particle save the position and minimum function value found by that particle $(x_{1,\min}, \cdots, x_{n,\min})$ and $(f(x_{1,\min}), \cdots, f(x_{n,\min}))$.
3. Save the global position and minimum function value $x_\min$ and $f(x_\min)$.
4. Initialize a counter $k=1$ used to track the iteration number.
5. Update the velocity of each particle for the current iteration $v_{i,k} = \omega v_{i,k-1} + p_1 r_{i,1} (x_{i,\min} - x_{i,k-1}) + p_2 r_{i,2} (x_\min - x_{i,k-1})$ where $\omega$ is a inertia constant, $p_1, p_2$ are momentum constants, and $r_{i,1}, r_{i,2}$ are per-particle random numbers in the range $[0,1]$.
6. Update the position of each particle for the current iteration $x_{i,k} = x_{i,k-1} + v_{i,k}$.
7. Evaluate the objective function at each new position and update the per-particle and global minimum function values and position.
8. Repeat from step 4 until the number of iterations are reached.

### References
>  J. Kennedy, R. C. Eberhart, and Y. Shi, Swarm Intelligence. Morgan Kaufmann, 2001. 

> Mykel J. Kochenderfer and Tim A. Wheeler. 2019. Algorithms for Optimization. The MIT Press.

In [1]:
import numpy as np

## Particle Swarm Method

In [2]:
def particle_swarm(fx, x0s, omega, p1, p2, bounds, niter):
    """
    particle_swarm returns the point xk where fx is minimum

    Parameters
    ----------
    fx : function
        function to minimize
    x0s : numpy.ndarray
        initial positions of particles in swarm
    omega : float
        inertia coefficient
    p1 : float
        momentum coefficient towards min position of current particle
    p2 : float
        momentum coefficient towards min position among all particles
    bounds : numpy.ndarray
        domain boundaries [x1_min, x1_max, ..., xn_min, xn_max]
    niter : int
        number of iterations

    Returns
    -------
    numpy.ndarray
        point xk where fx is minimum
    numpy.ndarray
        current and minimum position and value history for each particle
        [[x_1,0, fx(x_1,0), xk_1,min, fx(xk_1,min),...,
            x_n,0, fx(x_n,0), xk_n,min, fx(xk_n,min)],
         [x_1,1, fx(x_1,1), xk_1,min, fx(xk_1,min),...,
            x_n,1, fx(x_n,1), xk_n,min, fx(xk_n,min)],
    """

    # Initialize swarm with position, velocity, and min position.
    pos = np.copy(x0s)
    x0sdelta = np.max(x0s, axis=0) - np.min(x0s, axis=0)
    vel = (np.random.random(x0s.shape)-0.5)*x0sdelta
    posmin, fxmin = np.copy(x0s), np.apply_along_axis(fx, 1, x0s)

    # Global minimum position.
    xk_min, fxk_min = posmin[np.argmin(fxmin),:], np.min(fxmin)

    # Save position, velocity, and min position by particle to history.
    npart, ndim = x0s.shape[0], x0s.shape[1]
    steps = np.zeros((npart*niter, ndim*3+1))
    steps[:npart,:] = np.hstack((pos, vel, posmin, fxmin[:,np.newaxis]))

    # Perform fixed number of iterations.
    for k in range(1,niter):
        
        # Compute new velocity of each particle.
        rs = np.random.random((npart,2))
        vel = omega*vel + p1*rs[0]*(posmin-pos) + p2*rs[1]*(xk_min-pos)

        # Update the position of each particle based on velocity.
        pos = pos + vel
        pos = np.clip(pos, a_min=bounds[::2], a_max=bounds[1::2])

        # Evaluate the objective function at each new position.
        fxpart = np.apply_along_axis(fx, 1, pos)
        
        # If objective function is improved, 
        # then replace particle minimum position and value.
        inds = fxpart < fxmin
        posmin[inds,:], fxmin[inds] = pos[inds,:], fxpart[inds]

        # If global objective function is improved, 
        # then replace global minimum position and value.
        ind = np.argmin(fxmin)
        if fxmin[ind] < fxk_min:
            xk_min, fxk_min = posmin[ind,:], fxmin[ind]

        # Save particle history.
        ind0 = k*npart
        steps[ind0:ind0+npart,:] = np.hstack((pos, vel, posmin,
                                              fxmin[:,np.newaxis]))

    return xk_min, steps

## Test Function: Rosenbrock Function

In [3]:
def rosenbrock(x):
    """
    rosenbrock evaluates Rosenbrock function at vector x

    Parameters
    ----------
    x : array
        x is a D-dimensional vector, [x1, x2, ..., xD]

    Returns
    -------
    float
        scalar result
    """
    D = len(x)
    i, iplus1 = np.arange(0,D-1), np.arange(1,D)
    return np.sum(100*(x[iplus1] - x[i]**2)**2 + (1-x[i])**2)

## Solution To Rosenbrock Function

In [4]:
fx = rosenbrock
# Initial position of particles across domain.
# NOTE(mmorais): Deliberately avoid points near global minimum.
x0s = np.array([[-1.,-1.],[1.,-1.],[-1.,1.],[0.,1.5]])
niter = 5000
omega, p1, p2 = 1., 1., 1.
bounds = np.array([-2.,2.,-2.,2.])
xk, steps = particle_swarm(fx, x0s, omega, p1, p2, bounds, niter)

# Extract the initial position having the minimum value.
x0 = steps[np.argmin(steps[:x0s.shape[0], -1]),:2]

print("x0               :", x0)
print("rosenbrock f(w0) :", rosenbrock(x0))
print("----------------------------------")
print("xk               :", xk)
print("rosenbrock f(xk) :", rosenbrock(xk))
print("nsteps           :", niter)

x0               : [-1.  1.]
rosenbrock f(w0) : 4.0
----------------------------------
xk               : [0.99429233 0.98952196]
rosenbrock f(xk) : 0.00011442825405660791
nsteps           : 5000


## Test Function: Goldstein-Price Function

In [5]:
def goldstein_price(x):
    """
    goldstein_price evaluates Goldstein-Price function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    a = (x[0] + x[1] + 1)**2
    b = 19 - 14*x[0] + 3*x[0]**2 - 14*x[1] + 6*x[0]*x[1] + 3*x[1]**2
    c = (2*x[0] - 3*x[1])**2
    d = 18 - 32*x[0] + 12*x[0]**2 + 48*x[1] - 36*x[0]*x[1] + 27*x[1]**2
    return (1. + a*b) * (30. + c*d)

## Solution to Goldstein-Price Function

In [6]:
fx = goldstein_price
# Initial position of particles across domain.
# NOTE(mmorais): Deliberately avoid points near global minimum.
x0s = np.array([[-1.5,1.],[0.,1.5],[1.5,1.5],[-1.5,-1.],[1.,0.]])
niter = 5000
omega, p1, p2 = 1., 1., 1.
bounds = np.array([-2.,2.,-2.,2.])
xk, steps = particle_swarm(fx, x0s, omega, p1, p2, bounds, niter)

# Extract the initial position having the minimum value.
x0 = steps[np.argmin(steps[:x0s.shape[0], -1]),:2]

print("x0                    :", x0)
print("goldstein_price f(w0) :", goldstein_price(x0))
print("----------------------------------")
print("xk                    :", xk)
print("goldstein_price f(xk) :", goldstein_price(xk))
print("nsteps                :", niter)

x0                    : [1. 0.]
goldstein_price f(w0) : 726.0
----------------------------------
xk                    : [-0.01133158 -0.99957836]
goldstein_price f(xk) : 3.0333595279873276
nsteps                : 5000


## Test Function: Bartels-Conn Function

In [7]:
def bartels_conn(x):
    """
    bartels_conn evaluates Bartels-Conn function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    a = np.abs(x[0]**2 + x[1]**2 + x[0]*x[1])
    b = np.abs(np.sin(x[0]))
    c = np.abs(np.cos(x[1]))
    return a + b +c

## Solution to Bartels-Conn Function

In [8]:
fx = bartels_conn
# Initial position of particles across domain.
# NOTE(mmorais): Deliberately avoid points near global minimum.
x0s = np.array([[-3.,3.],[0.,3.],[3.,3.],
                [-3.,-3.],[0.,-3.],[3.,-3]])
niter = 5000
omega, p1, p2 = 1., 1., 1.
bounds = np.array([-5.,5.,-5.,5.])
xk, steps = particle_swarm(fx, x0s, omega, p1, p2, bounds, niter)

# Extract the initial position having the minimum value.
x0 = steps[np.argmin(steps[:x0s.shape[0], -1]),:2]

print("x0                 :", x0)
print("bartels_conn f(w0) :", bartels_conn(x0))
print("----------------------------------")
print("xk                 :", xk)
print("bartels_conn f(xk) :", bartels_conn(xk))
print("nsteps             :", niter)

x0                 : [0. 3.]
bartels_conn f(w0) : 9.989992496600445
----------------------------------
xk                 : [-0.00095064  0.10885384]
bartels_conn f(xk) : 1.0067784905734738
nsteps             : 5000


## Test Function: Egg Crate Function

In [9]:
def egg_crate(x):
    """
    egg_crate evaluates Egg Crate function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    return x[0]**2 + x[1]**2 + 25.*(np.sin(x[0])**2 + np.sin(x[1])**2)

## Solution to Egg Crate Function

In [10]:
fx = egg_crate
# Initial position of particles across domain.
# NOTE(mmorais): Deliberately avoid points near global minimum.
x0s = np.array([[-3.,3.],[0.,3.],[3.,3.],
                [-3.,-3.],[0.,-3.],[3.,-3]])
niter = 5000
omega, p1, p2 = 1., 1., 1.
bounds = np.array([-5.,5.,-5.,5.])
xk, steps = particle_swarm(fx, x0s, omega, p1, p2, bounds, niter)

# Extract the initial position having the minimum value.
x0 = steps[np.argmin(steps[:x0s.shape[0], -1]),:2]

print("x0              :", x0)
print("egg_crate f(w0) :", egg_crate(x0))
print("----------------------------------")
print("xk              :", xk)
print("egg_crate f(xk) :", egg_crate(xk))
print("nsteps          :", niter)

x0              : [0. 3.]
egg_crate f(w0) : 9.497871416870424
----------------------------------
xk              : [-0.02350755  0.00138511]
egg_crate f(xk) : 0.01441506173272409
nsteps          : 5000
